In [142]:
# import pandas
import pandas as pd
# import urllib to get data from webpage
from urllib.request import Request, urlopen
# import beautifulsoup, regular expression to extract data
from bs4 import BeautifulSoup as bs
import re
import os

In [143]:
# Read the current cleaned birds data to get the idx
birds_df = pd.read_csv("birds_April_13.csv")
birds_df.set_index('id', inplace=True)

In [144]:
birds_df['common_name_in_url'] = birds_df['common_name'].map(lambda x:"_".join(x.split()))
url_prefix = "https://en.wikipedia.org/wiki/"
birds_df['url'] = url_prefix + birds_df['common_name_in_url']

In [145]:
imgs_df = pd.DataFrame()
imgs_df['id'] = pd.np.nan
imgs_df['filename'] = pd.np.nan
imgs_df['url'] = pd.np.nan
imgs_df['source'] = pd.np.nan
imgs_df['author'] = pd.np.nan

In [146]:
imgs_cannot_download = []

In [147]:
def down_one(soup, idx):
    number_of_noviewer = len(soup.find_all('img', { "class" : "noviewer" }))
    remove_number_from_end = - (3 + number_of_noviewer)
    img_tags = soup.find_all('img')[:remove_number_from_end]
    count = 0
    for tag in img_tags:
        src = tag['src']
        if 'Status' in src or 'map' in src or 'Icon' in src:
            continue
        try:
            skip_to_url = 'https://en.wikipedia.org' + tag.parent['href']
        except:
            continue
        img_link = 'https:' + src.replace('/thumb', '')
        # print(img_link)
        img_link = re.search('.*?.(jpg|JPG)', img_link)
        if img_link is None:
            continue
        img_link = img_link[0]
        count = count + 1
        filename = 'hdimgs/' + 'hd' + str(idx) + '_' + str(count) + '.jpg'
        if os.path.isfile(filename):
            continue
        try:
            request = urlopen(img_link, timeout=5)
            with open(filename, 'wb') as f:
                f.write(request.read())
        except:
            imgs_cannot_download.add(skip_to_url)
        author = get_author(skip_to_url, idx)
        if not author:
            continue
        new_record = [idx, filename, img_link, author[0], author[1]]
        imgs_df = imgs_df.append(pd.DataFrame([new_record], columns=imgs_df.columns))

In [148]:
def get_author(url, idx):
    try:
        df = pd.read_html(skip_to_url)
    except:
        return False
    df = df[1]
    img_source = False
    img_author = False
    for row in df.index:
        if df[0][row] == 'Source':
            img_source = df[1][row]
        if df[0][row] == 'Author':
            img_author = df[1][row]
            break
    if img_author or img_source is False:
        return False
    return img_source, img_author

In [149]:
for idx in birds_df.index:
    url = birds_df.ix[idx, 'url']
    # use Firefox as an user-agent, otherwise the connection will be rejected.
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    try:
        webpage = urlopen(req).read()
    except:
        birds_df.ix[idx, 'url'] = pd.np.nan
        continue
    # get the content of a webpage
    webpage = urlopen(req).read()
    # convert to string
    webpage_str = str(webpage)
    # initialize a beautiful soup with lxml engine
    soup = bs(webpage, "lxml")
    down_one(soup, idx)

AttributeError: 'list' object has no attribute 'add'

In [17]:
birds_list.to_csv("birds_info_wiki.csv")

In [22]:
birds_list.ix[39, 'url'] = 'https://en.wikipedia.org/wiki/Mallard'

In [26]:
birds_list.ix[354, 'url'] = 'https://en.wikipedia.org/wiki/Baird%27s_sandpiper'

In [39]:
birds_list = pd.concat([birds_list, pd.DataFrame(columns=['nsw', 'vic', 'tas', 'act', 'sa', 'wa', 'nt', 'qld', 
                                                          'min_weight', 'max_weight', 'min_length', 'max_length',
                                                          'images_url', 'voice_url', 'Description', 'Sources'
                                                          'colour'])])

In [40]:
birds_list_nul = birds_list[birds_list['url'].isnull()]

In [41]:
len(birds_list)

658

In [42]:
len(birds_list_nul)

1

In [43]:
birds_list_nul

,Binomial,Category,Common name,Family,Notes,Order,colour,common_name_in_url,size,url,...,nt,qld,min_weight,max_weight,min_length,max_length,images_url,voice_url,Description,Sourcescolour
884,Motacilla taivana,Wagtails and pipits,Green-headed yellow wagtail,Motacillidae,NaN,Passeriformes,NaN,Green-headed_yellow_wagtail,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
birds_list.to_csv("birds_info_wiki.csv")

In [45]:
birds_list_nul.to_csv('birds_not_found_in_wiki.csv')